In [97]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

import pandas as pd
import os

In [5]:
MASTER_IP_ADDRESS="spark://192.168.140.136:7077"
APP_NAME="recommender"

# Init:
## I. TODO: 
.config("spark.cores.max","4")\
.config("spark.executor.cores", "2")\

In [11]:
spark=SparkSession.builder\
.master(MASTER_IP_ADDRESS)\
.appName(APP_NAME)\
.getOrCreate()

In [19]:
sc=spark.sparkContext

In [17]:
spark

# II. Samples: 

In [75]:
ds = range(1,4)
rdd = sc.parallelize(ds)
# rdd=rdd.map(lambda x: (x, "a" * x))
rdd.collect()

[1, 2, 3]

In [77]:
# PATH_DS="file:///..."
PATH_DS="../ds"
rdd.saveAsTextFile(PATH_DS)

In [92]:
read_rdd = sc.textFile(PATH_DS)
read_rdd.collect()

['2', '3', '1']

In [93]:
read_rdd=read_rdd.map(lambda x: int(x)*2)
sorted(read_rdd.collect())

[2, 4, 6]

21/11/27 22:09:30 ERROR TaskSchedulerImpl: Lost executor 0 on 192.168.140.136: worker lost
21/11/27 22:09:31 ERROR TaskSchedulerImpl: Lost executor 3 on 192.168.140.136: worker lost


# III. Load data:

In [109]:
PATH_DB="../db"
PATH_DB_MOVIE=f"{PATH_DB}/movie.csv"
PATH_DB_RATING=f"{PATH_DB}/rating_batch_001.csv"

In [100]:
os.listdir(PATH_DB)

['genome_tags.csv',
 'genome_scores.csv',
 'movie.csv',
 'link.csv',
 'rating.csv',
 'tag.csv']

In [196]:
'''
- https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html
'''

movie = pd.read_csv(PATH_DB_MOVIE)
movie = movie.loc[:,["movieId","title"]]

# movie = spark.read.format("csv").option("header", "true").load(PATH_DB_MOVIE)
# movie=movie.select("movieId","title")
movie.head(10)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


In [197]:
rating = pd.read_csv(PATH_DB_RATING)
rating = rating.loc[:,["userId","movieId","rating"]]

# rating = spark.read.format("csv").option("header", "true").load(PATH_DB_RATING)
# rating=movie.select("userId","movieId","rating")
rating.head(10)

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5
5,1,112,3.5
6,1,151,4.0
7,1,223,4.0
8,1,253,4.0
9,1,260,4.0


In [198]:
ds = pd.merge(movie,rating)
# ds=movie.join(rating, ["movieId"], 'full')
ds.head(10)

,movieId,title,userId,rating
0,1,Toy Story (1995),3,4.0
1,1,Toy Story (1995),6,5.0
2,1,Toy Story (1995),8,4.0
3,1,Toy Story (1995),10,4.0
4,1,Toy Story (1995),11,4.5
5,2,Jumanji (1995),1,3.5
6,2,Jumanji (1995),5,3.0
7,3,Grumpier Old Men (1995),2,4.0
8,3,Grumpier Old Men (1995),6,3.0
9,3,Grumpier Old Men (1995),7,3.0


In [201]:
ds_pivot_table = ds.pivot_table(index = ["userId"],columns = ["title"],values = "rating")
ds_pivot_table.head(10)

title,1984 (Nineteen Eighty-Four) (1984),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),28 Days (2000),28 Days Later (2002),"7th Voyage of Sinbad, The (1958)",8MM (1999),A.I. Artificial Intelligence (2001),Abbott and Costello Meet Frankenstein (1948),"Abyss, The (1989)",...,"Wizard of Oz, The (1939)","Wolf Man, The (1941)","X-Files: Fight the Future, The (1998)",X-Men (2000),X2: X-Men United (2003),Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,3.5,4.0,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,4.0,NaN,3.0,NaN,4.0,NaN
2,NaN,5.0,NaN,3.0,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,5.0,4.0,NaN,NaN,NaN,4.0,NaN,NaN,3.0,...,4.0,NaN,5.0,NaN,NaN,3.0,NaN,NaN,5.0,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,3.0,3.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,4.0,NaN,4.0,NaN,NaN,NaN,3.0,NaN,2.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [226]:
ts=spark.createDataFrame(ds_pivot_table)

In [227]:
'''
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corrwith.html
'''
# movie_watched = pivot_table["Wizard of Oz, The (1939)"]
# similarity_with_other_movies = pivot_table.corrwith(movie_watched) 
# similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=True)
# similarity_with_other_movies.head(20)

ts.select(ts.columns[0]).show()

+----------------------------------+
|1984 (Nineteen Eighty-Four) (1984)|
+----------------------------------+
|                               NaN|
|                               NaN|
|                               5.0|
|                               NaN|
|                               NaN|
|                               NaN|
|                               NaN|
|                               NaN|
|                               NaN|
|                               NaN|
|                               NaN|
+----------------------------------+



In [228]:
# ts=ts.replace(float("NaN"), 0)

In [229]:
ts.select(ts.columns[0]).show()

21/11/28 01:39:25 ERROR TransportClient: Failed to send RPC RPC 8367316275359843451 to /192.168.140.136:48378: io.netty.channel.StacklessClosedChannelException
io.netty.channel.StacklessClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(Object, ChannelPromise)(Unknown Source)
21/11/28 01:39:25 WARN BlockManagerMasterEndpoint: Error trying to remove broadcast 297 from block manager BlockManagerId(3, 192.168.140.136, 33421, None)
java.io.IOException: Failed to send RPC RPC 8367316275359843451 to /192.168.140.136:48378: io.netty.channel.StacklessClosedChannelException
	at org.apache.spark.network.client.TransportClient$RpcChannelListener.handleFailure(TransportClient.java:392)
	at org.apache.spark.network.client.TransportClient$StdChannelListener.operationComplete(TransportClient.java:369)
	at io.netty.util.concurrent.DefaultPromise.notifyListener0(DefaultPromise.java:578)
	at io.netty.util.concurrent.DefaultPromise.notifyListenersNow(DefaultPromise.java:552)

+----------------------------------+
|1984 (Nineteen Eighty-Four) (1984)|
+----------------------------------+
|                               NaN|
|                               NaN|
|                               5.0|
|                               NaN|
|                               NaN|
|                               NaN|
|                               NaN|
|                               NaN|
|                               NaN|
|                               NaN|
|                               NaN|
+----------------------------------+



In [ ]:
Correlation.corr(ts, "1984 (Nineteen Eighty-Four) (1984)", 'pearson')

In [232]:
from pyspark.ml.linalg import DenseMatrix, Vectors
from pyspark.ml.stat import Correlation
dataset = [[Vectors.dense([1, 0, 0, -2])],
           [Vectors.dense([4, 5, 0, 3])],
           [Vectors.dense([6, 7, 0, 8])],
           [Vectors.dense([9, 0, 0, 1])]]
dataset = spark.createDataFrame(dataset, ['features'])
pearsonCorr = Correlation.corr(dataset, 'features', 'pearson').collect()[0][0]
# print(str(pearsonCorr).replace('nan', 'NaN'))




# spearmanCorr = Correlation.corr(dataset, 'features', method='spearman').collect()[0][0]
# print(str(spearmanCorr).replace('nan', 'NaN'))


21/11/28 01:42:21 ERROR TransportClient: Failed to send RPC RPC 6081302213560872951 to /192.168.140.136:48378: io.netty.channel.StacklessClosedChannelException
io.netty.channel.StacklessClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(Object, ChannelPromise)(Unknown Source)
21/11/28 01:42:21 WARN BlockManagerMasterEndpoint: Error trying to remove broadcast 326 from block manager BlockManagerId(3, 192.168.140.136, 33421, None)
java.io.IOException: Failed to send RPC RPC 6081302213560872951 to /192.168.140.136:48378: io.netty.channel.StacklessClosedChannelException
	at org.apache.spark.network.client.TransportClient$RpcChannelListener.handleFailure(TransportClient.java:392)
	at org.apache.spark.network.client.TransportClient$StdChannelListener.operationComplete(TransportClient.java:369)
	at io.netty.util.concurrent.DefaultPromise.notifyListener0(DefaultPromise.java:578)
	at io.netty.util.concurrent.DefaultPromise.notifyListenersNow(DefaultPromise.java:552)

21/11/28 01:42:22 ERROR TransportClient: Failed to send RPC RPC 5382459686187599373 to /192.168.140.136:48378: io.netty.channel.StacklessClosedChannelException
io.netty.channel.StacklessClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(Object, ChannelPromise)(Unknown Source)
21/11/28 01:42:22 WARN BlockManagerMasterEndpoint: Error trying to remove broadcast 336 from block manager BlockManagerId(3, 192.168.140.136, 33421, None)
java.io.IOException: Failed to send RPC RPC 5382459686187599373 to /192.168.140.136:48378: io.netty.channel.StacklessClosedChannelException
	at org.apache.spark.network.client.TransportClient$RpcChannelListener.handleFailure(TransportClient.java:392)
	at org.apache.spark.network.client.TransportClient$StdChannelListener.operationComplete(TransportClient.java:369)
	at io.netty.util.concurrent.DefaultPromise.notifyListener0(DefaultPromise.java:578)
	at io.netty.util.concurrent.DefaultPromise.notifyListenersNow(DefaultPromise.java:552)

In [ ]:
'''
TODO: 
1. Theories: Correlation (Pairwise) - https://www.kaggle.com/kanncaa1/recommendation-systems-tutorial/notebook
2. Understand Kaggle sample.
3. Understand PySpark API. 
4. Implementation.
'''